# Pandera

[Pandera](https://pandera.readthedocs.io/en/stable/index.html) is an Open-source Framework for Data Validation

This notebook shows a brief example


In [1]:
import pandas as pd
import pandera as pa
from pandera.engines import pandas_engine

## I. Get the data from CSV 

The data consist of 3 tables:
 - **super_regions**
 - **regions** : which are subdivisions of super_regions
 - **power_load** : electric power load for a given date and given region

### I.1 Helper "get" functions 

In [2]:
def get_super_regions():
    return pd.read_csv('data/super_regions.csv', dtype={'super_region': 'string'}, sep = ',')

def get_regions():
    return pd.read_csv('data/regions.csv', dtype={'super_region': 'string', 'region': 'string'}, sep = ',')

def get_power_load():
    return pd.read_csv('data/power_load.csv', dtype={'region': 'string', 'load': 'float64'}, sep = ',', parse_dates=['date'])

### I.2 Get the data

#### I.2.a Super Regions

In [3]:
super_regions = get_super_regions()
super_regions.head()

,super_region
0,Upstate
1,Downstate


#### I.2.b Regions

In [4]:
regions = get_regions()
regions.head()

,region,super_region
0,Capital,Upstate
1,Central,Upstate
2,Dunwoodie,Upstate
3,Genesee,Upstate
4,Hudson Valley,Downstate


In [5]:
power_load = get_power_load()
power_load.head()

,region,date,load
0,Capital,2018-06-20,1495.583333
1,Capital,2018-06-21,1470.291667
2,Capital,2018-06-22,1422.458333
3,Capital,2018-06-23,1357.291667
4,Capital,2018-06-24,1301.416667


### I.3 Show the CSV data structures

#### I.2.a Super Regions

In [6]:
super_regions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   super_region  2 non-null      string
dtypes: string(1)
memory usage: 148.0 bytes


#### I.2.b Regions

In [7]:
regions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   region        11 non-null     string
 1   super_region  11 non-null     string
dtypes: string(2)
memory usage: 308.0 bytes


#### I.2.c Power Load

In [8]:
power_load.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18216 entries, 0 to 18215
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   region  18216 non-null  string        
 1   date    18216 non-null  datetime64[ns]
 2   load    17818 non-null  float64       
dtypes: datetime64[ns](1), float64(1), string(1)
memory usage: 427.1 KB


## II. Validating data

### II.1 Method #1 : DataFrame Schemas

_"The DataFrameSchema class enables the specification of a schema that verifies the columns and index of a pandas DataFrame object."_

#### II.1.a Set up the schemas & create an example

In [9]:
super_regions_schema = pa.DataFrameSchema(
    {
        "super_region": pa.Column(str, nullable=False),
    },
    unique=["super_region"],
    strict=True,
)
super_regions_schema.example(size=1)

,super_region
0,`T񦞨ý


In [10]:
regions_schema = pa.DataFrameSchema(
    {
        "region": pa.Column(str, nullable=False),
        "super_region": pa.Column(str, nullable=False),
    },
    unique=["region", "super_region"],
    strict=True,
)
regions_schema.example(size=1)

,region,super_region
0,򥼾󘤳燎¾񞠼f\t,򥼾󘤳燎¾񞠼f\t


In [11]:
power_load_schema = pa.DataFrameSchema(
    {
        "region": pa.Column(str, nullable=False),
#        "date": pa.Column("datetime64", nullable=False),
        "date": pa.Column(pandas_engine.DateTime(to_datetime_kwargs = {"format":"%Y/%m/%d"}), nullable=False),
        "load": pa.Column(float, checks=pa.Check.gt(0), nullable=False),
    },
    unique=["region", "date"],
    strict=True,
)
power_load_schema.example(size=1)


,region,date,load
0,,1970-01-01 00:00:00.000000011,4.940656e-324


#### II.1.b Create a validation helper function

In [12]:
def validate_schema(df: pd.DataFrame, df_schema: pa.DataFrameSchema, msg: str) -> pd.DataFrame:
    try:
        print(msg, '...', sep='')
        validated_df = df_schema.validate(df)
        print(msg, '...OK', sep='')
        return(validated_df)
    except pa.errors.SchemaError as e:
        print(msg, '...FAILED =>', sep='')
        print("****************************************")
        print(e)
        print("****************************************")

#### II.1.c Validate the data

In [13]:
validated_super_regions = validate_schema(super_regions, super_regions_schema, "Validating super_regions")
validated_super_regions.head()

Validating super_regions...
Validating super_regions...OK


,super_region
0,Upstate
1,Downstate


In [14]:

validated_regions = validate_schema(regions, regions_schema, "Validating regions")
validated_regions.head()

Validating regions...
Validating regions...OK


,region,super_region
0,Capital,Upstate
1,Central,Upstate
2,Dunwoodie,Upstate
3,Genesee,Upstate
4,Hudson Valley,Downstate


In [15]:
# print(power_load[(power_load['region'] == 'Capital') & (power_load['date'] == '2018/07/31')])
# power_load.loc[(power_load['region'] == 'Capital') & (power_load['date'] == '2018/07/31'), 'load'] = np.nan
# print(power_load[(power_load['region'] == 'Capital') & (power_load['date'] == '2018/07/31')])

validated_power_load = validate_schema(power_load, power_load_schema, "Validating power_load")

Validating power_load...
Validating power_load...FAILED =>
****************************************
non-nullable series 'load' contains null values:
41      NaN
47      NaN
75      NaN
132     NaN
223     NaN
         ..
17373   NaN
17486   NaN
17631   NaN
17843   NaN
18211   NaN
Name: load, Length: 398, dtype: float64
****************************************


The power_load validation **fails** due to missing load values.  

Remove the NAs and re-validate.

In [16]:
power_load_no_NAs = power_load[(power_load['load'].notna())]
validated_power_load = validate_schema(power_load_no_NAs, power_load_schema, "Validating power_load_no_NAs")

Validating power_load_no_NAs...
Validating power_load_no_NAs...FAILED =>
****************************************
Column 'load' failed element-wise validator number 0: greater_than(0) failure cases: -10953.3650572372, -18691.4685346976, -20877.2361447536, -24327.9417834871, -8131.50693110981, -14596.6955819775, -4068.32223664545, -9312.78214695918, -14656.4075716635, -5426.63989076665, -1728.57859662086, -13960.6625610962, -1652.15154619529, -3364.33682026991, -2143.82004485823, -8601.85849590299, -7848.73712505549, -2399.54324270216, -3805.30752932024, -1848.30533380647, -2976.88738208294, -11216.5685038337, -407.73131766232, -7785.16881659046, -2586.8434567858, -784.655206493677, -6015.13141964274, -6310.4687102185, -2286.80814656715, -26010.7894310697, -11099.8920933817, -13017.9062438803
****************************************


The power_load validation still **fails** due to negative load values: a power_load cannot be negative or null.  

Remove the negative values & re-validate.

In [17]:
power_load_no_NAs_no_negatives = power_load_no_NAs[(power_load_no_NAs['load'] > 0)]
validated_power_load = validate_schema(power_load_no_NAs_no_negatives, power_load_schema, "Validating power_load_no_NAs_no_negatives")

Validating power_load_no_NAs_no_negatives...
Validating power_load_no_NAs_no_negatives...OK


### II.2 Method #2 : DataFrame Model

_"pandera also provides an alternative API for expressing schemas inspired by dataclasses and pydantic."_  
_"pandera provides a class-based API that’s heavily inspired by pydantic. In contrast to the object-based API, you can define dataframe models in much the same way you’d define pydantic models."_  
_"DataFrameModel s are annotated with the pandera.typing module using the standard typing syntax. Models can be explicitly converted to a DataFrameSchema or used to validate a DataFrame directly."_

#### II.2.a : Create the data models & an example

In [18]:
class SuperRegionsDataFrameModel(pa.DataFrameModel):
    super_region: str = pa.Field(nullable=False)

SuperRegionsDataFrameModel.example(size=1)

,super_region
0,<&


In [19]:
class RegionsDataFrameModel(pa.DataFrameModel):
    super_region: str = pa.Field(nullable=False)
    region: str = pa.Field(nullable=False)

RegionsDataFrameModel.example(size=1)

,super_region,region
0,8,8


In [20]:
class PowerLoadDataFrameModel(pa.DataFrameModel):
        region: str = pa.Field(nullable=False)
        date: pandas_engine.DateTime(to_datetime_kwargs = {"format":"%Y/%m/%d"}) = pa.Field(nullable=False)
        load: float  = pa.Field(gt=0, nullable=False)

PowerLoadDataFrameModel.example(size=1)

,region,date,load
0,Å¯,1969-12-31 23:59:59.999937938,1.175494e-38


#### II.2.b Create a validation helper function

In [21]:
def validate_dataframe_model(df: pd.DataFrame, df_model: pa.DataFrameModel, msg: str) -> pd.DataFrame:
    try:
        print(msg, '...', sep='')
        validated_df = df_model.validate(df)
        print(msg, '...OK', sep='')
        return(validated_df)
    except pa.errors.SchemaError as e:
        print(msg, '...FAILED =>', sep='')
        print("****************************************")
        print(e)
        print("****************************************")

#### II.2.c Validate the data

In [22]:
validated_super_regions = validate_dataframe_model(super_regions, SuperRegionsDataFrameModel, "Validating super_regions")
validated_super_regions.head()

Validating super_regions...
Validating super_regions...OK


,super_region
0,Upstate
1,Downstate


In [23]:
validated_regions = validate_dataframe_model(regions, RegionsDataFrameModel, "Validating regions")
validated_regions.head()

Validating regions...
Validating regions...OK


,region,super_region
0,Capital,Upstate
1,Central,Upstate
2,Dunwoodie,Upstate
3,Genesee,Upstate
4,Hudson Valley,Downstate


In [24]:
validated_power_load = validate_dataframe_model(power_load, PowerLoadDataFrameModel, "Validating load_data")

Validating load_data...
Validating load_data...FAILED =>
****************************************
non-nullable series 'load' contains null values:
41      NaN
47      NaN
75      NaN
132     NaN
223     NaN
         ..
17373   NaN
17486   NaN
17631   NaN
17843   NaN
18211   NaN
Name: load, Length: 398, dtype: float64
****************************************


Same as before: the validation **fails** due to null values.  

Validate again with: power_load_no_NAs.

In [25]:
validated_power_load = validate_dataframe_model(power_load_no_NAs, PowerLoadDataFrameModel, "Validating power_load_no_NAs")

Validating power_load_no_NAs...
Validating power_load_no_NAs...FAILED =>
****************************************
Column 'load' failed element-wise validator number 0: greater_than(0) failure cases: -10953.3650572372, -18691.4685346976, -20877.2361447536, -24327.9417834871, -8131.50693110981, -14596.6955819775, -4068.32223664545, -9312.78214695918, -14656.4075716635, -5426.63989076665, -1728.57859662086, -13960.6625610962, -1652.15154619529, -3364.33682026991, -2143.82004485823, -8601.85849590299, -7848.73712505549, -2399.54324270216, -3805.30752932024, -1848.30533380647, -2976.88738208294, -11216.5685038337, -407.73131766232, -7785.16881659046, -2586.8434567858, -784.655206493677, -6015.13141964274, -6310.4687102185, -2286.80814656715, -26010.7894310697, -11099.8920933817, -13017.9062438803
****************************************


Same as before: the validation **fails again** due to negative values.  

Validate again with: power_load_no_NAs_no_nagatives

In [26]:
validated_power_load = validate_dataframe_model(power_load_no_NAs_no_negatives, PowerLoadDataFrameModel, "Validating power_load_no_NAs_no_negatives")

Validating power_load_no_NAs_no_negatives...
Validating power_load_no_NAs_no_negatives...OK
